In [1]:
#Import library
import tensorflow as tf
from google.colab import drive

In [2]:
#Import dataset from google drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
%cd /content/drive/My\ Drive/Colab\ Storage\ Real

/content/drive/My Drive/Colab Storage Real


In [10]:
#Build model
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [11]:
#Compile model
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'train',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        'val',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 4192 images belonging to 2 classes.
Found 1057 images belonging to 2 classes.


In [14]:
checkpoint_path = "checkpoint_coursera/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=10 * len(train_generator))

In [15]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=100,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8, callbacks = [cp_callback])

Epoch 1/100
8/8 [==============================] - 435s 54s/step - loss: 0.6251 - accuracy: 0.6663 - val_loss: 0.5926 - val_accuracy: 0.7344
Epoch 2/100
8/8 [==============================] - 338s 42s/step - loss: 0.5621 - accuracy: 0.7568 - val_loss: 0.5424 - val_accuracy: 0.7852
Epoch 3/100
8/8 [==============================] - 249s 31s/step - loss: 0.5903 - accuracy: 0.7227 - val_loss: 0.6010 - val_accuracy: 0.7422
Epoch 4/100
8/8 [==============================] - 182s 23s/step - loss: 0.5501 - accuracy: 0.7422 - val_loss: 0.5757 - val_accuracy: 0.7617
Epoch 5/100
8/8 [==============================] - 157s 20s/step - loss: 0.5448 - accuracy: 0.7324 - val_loss: 0.5822 - val_accuracy: 0.7188
Epoch 6/100
8/8 [==============================] - 107s 13s/step - loss: 0.5089 - accuracy: 0.7591 - val_loss: 0.5996 - val_accuracy: 0.7148
Epoch 7/100
8/8 [==============================] - 97s 12s/step - loss: 0.4926 - accuracy: 0.7578 - val_loss: 0.5148 - val_accuracy: 0.7734
Epoch 8/100
8/

In [16]:
model.save('coursera_model.h5')

In [19]:
# Creating test image flow

test_gen = ImageDataGenerator(rescale = 1./255)

test_set = test_gen.flow_from_directory('test',
                                        target_size = (300,
                                                       300),
                                        batch_size = 1,
                                        class_mode = None, shuffle=False)

# Making predictions

predictions = model.predict(test_set)
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1

Found 624 images belonging to 2 classes.


In [22]:
# Confusion matrix
import pandas as pd
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(data=confusion_matrix(test_set.classes, predictions, labels=[0, 1]),
                  index=["Actual Normal", "Actual Pneumonia"],
                  columns=["Predicted Normal", "Predicted Pneumonia"])
cm

,Predicted Normal,Predicted Pneumonia
Actual Normal,45,189
Actual Pneumonia,0,390
